In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import tensorflow as tf
import pathlib
from tqdm import trange

from src.DataPreprocessor.data_preprocessor import DataPreprocessor, Mode
from src.LearningKeras.net_architecture import cnn_150x150x5,cnn_150x150x5_3class
from src.LearningKeras.train import KerasTrainer

np.random.seed(1)
tf.set_random_seed(2)

check data_visualisation for details

In [2]:
from src.DataPreprocessor.DataIOBackend.gdal_backend import GdalBackend
from src.DataPreprocessor.data_visualiser import DataVisualiser

dataiobackend = GdalBackend()
data_preprocessor = DataPreprocessor(data_dir="data/Region 1 - Lopukangri/",
                              backend=dataiobackend,
                              filename_prefix="tibet",
                              mode=Mode.TRAIN,
                              seed=1)

## NN

### Train

In [3]:
model_generator = lambda: cnn_150x150x5_3class()
ensemble_size = 2
batch_size = 5

trainer = KerasTrainer(model_generator=model_generator,
                       ensemble_size=ensemble_size,
                       data_preprocessor=data_preprocessor,
                       batch_size=batch_size)
train_generator = data_preprocessor.train_generator(batch_size=batch_size,
                                         class_probabilities=np.array([1./3, 1./3, 1./3]),
                                         patch_size=(150, 150),
                                         channels=np.array([0, 1, 2, 3, 4]))

In [4]:
history_arr = trainer.train(steps_per_epoch=50, epochs=5, train_generator=train_generator)

Epoch 1/5
50/50 [==============================] - 71s 1s/step - loss: 1.0650 - acc: 0.4400 - val_loss: 0.8165 - val_acc: 0.6800
Epoch 2/5
50/50 [==============================] - 68s 1s/step - loss: 0.6989 - acc: 0.6680 - val_loss: 0.6361 - val_acc: 0.6400
Epoch 3/5
50/50 [==============================] - 71s 1s/step - loss: 0.7342 - acc: 0.6920 - val_loss: 0.9020 - val_acc: 0.6000
Epoch 4/5
50/50 [==============================] - 69s 1s/step - loss: 0.6677 - acc: 0.7200 - val_loss: 0.5870 - val_acc: 0.7200
Epoch 5/5
50/50 [==============================] - 71s 1s/step - loss: 0.5825 - acc: 0.7880 - val_loss: 0.4110 - val_acc: 0.8400
Epoch 1/5
50/50 [==============================] - 74s 1s/step - loss: 0.9920 - acc: 0.5720 - val_loss: 0.7061 - val_acc: 0.7200
Epoch 2/5
50/50 [==============================] - 70s 1s/step - loss: 0.7776 - acc: 0.6480 - val_loss: 0.6513 - val_acc: 0.6000
Epoch 3/5
50/50 [==============================] - 78s 2s/step - loss: 0.6782 - acc: 0.7160 - val

In [6]:
for i in range(ensemble_size):
    pathlib.Path('models_3class').mkdir(parents=True, exist_ok=True)
    trainer.models[i].save_weights('models_3class/model_{}.h5'.format(i))